In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import urllib
from dotenv import load_dotenv
import pandas as pd
from pprint import pprint
import requests
import os 
import base64
import json
from bs4 import BeautifulSoup

In [3]:
load_dotenv()

SpotifyClientID = os.getenv("SPOTIPY_CLIENT_ID")
SpotifyClientSecret = os.getenv("SPOTIPY_CLIENT_SECRET")
SpotifyRedirectURI = os.getenv("SPOTIPY_REDIRECT_URI")
SpotifyOath = os.getenv("SpotifyOath")

def _make_authorization_headers(client_id, client_secret):
    auth_header = base64.b64encode(
        f"{client_id}:{client_secret}".encode("ascii")
    )
    return {"Authorization": "Basic %s" % auth_header.decode("ascii")}

baseauth = _make_authorization_headers(SpotifyClientID, SpotifyClientSecret)

tok = requests.post(r"https://accounts.spotify.com/api/token", 
                   data={"grant_type": "client_credentials"},
                   headers=baseauth)
btok = json.loads(tok.content)['access_token']
bear = {"Authorization": f"Bearer {btok}"}

In [ ]:
search_str = 'Avatar Soundtrack'
options = 'album,playlist'
q = urllib.parse.quote(search_str)
o = urllib.parse.quote(options)
getme = f"https://api.spotify.com/v1/search?query={q}" \
    + f"&type={o}&market=US&offset=0&limit=10"
getme

In [ ]:
output = requests.get(getme, headers=bear)
jl = json.loads(output.content)
#alb = jl['albums']['items']
ply = jl['playlists']['items']
df = pd.json_normalize(alb).append(pd.json_normalize(ply))
df

In [ ]:
df.to_csv("check_avatar.csv", index=False)

In [ ]:
file = pd.read_csv("1990_2020_movie_list1227.csv", encoding='latin1')
titles = file.title.tolist()
movids = file.movieid.tolist()

searchbase = r"https://www.traileraddict.com/search/"
#IFERROR(VLOOKUP($G3, [trailer_search_testing.xlsx]trailer_search!$C$2:$R$206, COLUMN()-2,0), "")
#=VLOOKUP(B3780,trailer_search.csv!$G$1:$O$29, COLUMN()-5, 0)
res_list = []

for i,search_str in enumerate(titles):
    q = urllib.parse.quote(search_str).lower()
    url = searchbase + q
    #print(url)
    page = requests.get(url).text

    soup = BeautifulSoup(page, "html.parser")
    res = soup.find_all('div' , class_="info")

    for l in res:
        d = {'ttid': movids[i],
            'search': search_str,
             'href': l.find_previous_sibling()['href']}
        for ch in l.find_all('div',recursive=False):
            for cc in ch:
                if len(str(cc))>1:
                    ss = list(cc.stripped_strings)
                    if 'class' in cc.attrs.keys():
                        d[list(cc['class'])[0]] = ss[-1]
                    else:
                        d[ss[0]] = ss[-1]
        res_list.append(d)
print("we did it!")

In [ ]:
trail_df = pd.DataFrame(res_list)
trail_df.head()

In [ ]:
trail_df.shape

In [ ]:
# trail_df.to_csv("trailer_search.csv")

In [ ]:
import re 

trailers = pd.read_csv("1990_2020_movie_list_with_trailers.csv", encoding='latin1')
hrefs = trailers.href.tolist()

baseurl = r"https://www.traileraddict.com"

trailist = []
print(len(hrefs))
for i, h in enumerate(hrefs):
    print(i,h)
#h = r"/star-trek-into-darkness"
    page = requests.get(baseurl + h).text
    soup = BeautifulSoup(page, "html.parser")

    res = soup.find_all('div' , class_="featured_box") #, text=re.compile("(.*Trailer.*)"))
    for i,r in enumerate(res):
        if "Trailer" in r.text:
            me = {'href': h, 'enum': i}
            me['date'] = r.div.span.text
            me['link'] = r.div.a['href']
            me['title'] = r.div.a.text.strip()
            trailist.append(me)

In [ ]:
df_trail = pd.DataFrame(trailist)
df_trail.shape
df_trail.to_csv("trailer_links.csv")

In [ ]:
tlinks = pd.read_csv("trailer_links.csv", encoding='latin1')
tlinks = tlinks.query("getme == 'x'")
hrefs = tlinks.link.tolist()

baseurl = r"https://www.traileraddict.com"

trailist = []
print(len(hrefs))

# for i, h in enumerate(hrefs):
#     print(i,h)
h = r"/the-avengers/feature-trailer"
i = 0
page = requests.get(baseurl + h).text
soup = BeautifulSoup(page, "html.parser")

me = {'link': h, 'enum': i}
#me['mp4'] = soup.find('video')['src']

txts = soup.find('div', id='interior')

heads = ["Duration", "Views", "Posted On", "Cast", "Director", "Writer", "Studio", "Release", "Trailer Tracks"]

strs = txts.stripped_strings

curr_hd = ""
curr_list = []

for x in strs:
    if x in heads:
        if curr_hd != "":
            me[curr_hd] = ';'.join(curr_list)
        curr_hd = x
        curr_list = []
    else:
        curr_list.append(x)

trailist.append(me)

In [ ]:
df_trailers = pd.DataFrame(trailist)
df_trailers.shape
df_trailers.to_csv("trailer_details.csv")

In [22]:
#Selenium items
from selenium import webdriver #webdriver to control activities
from selenium.webdriver.common.by import By #types of navigation
from selenium.webdriver.support.ui import WebDriverWait #waiting
import time

def start_driver(runheadless = False):
    """Initialize a Chrome webdriver with which for Selenium to act. """
    #set Chromedriver options to work smoothly
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized") # open Browser in maximized mode
    options.add_argument("disable-infobars") # disabling infobars
    options.add_argument("--disable-extensions") # disabling extensions
    options.add_argument("--disable-gpu") # applicable to windows os only
    options.add_argument("--disable-dev-shm-usage") # overcome limited resource problems
    options.add_argument("--no-sandbox") # Bypass OS security model  
    if runheadless:
        options.add_argument("--headless")  
    options.add_argument("--remote-debugging-port=9222")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    driver = webdriver.Chrome(options=options,
            executable_path=r"D:\chromedriver.exe")
    driver.implicitly_wait(10) # seconds
    
    driver.get("https://www.google.com/") #open the given URL
    time.sleep(1) #wait for everything to load

    return driver

load_dotenv()


True

In [23]:
driver = start_driver()

In [33]:
tdicts = pd.read_csv("traileraddict_songlinks.csv", encoding='latin1')
tdicts = list(tdicts.T.to_dict().values())
tdicts[0]

{'imdb_id': 'tt0848228',
 'film_title': 'The Avengers',
 'year': 2012,
 'tradd_url': '/the-avengers',
 'link': '/the-avengers/feature-trailer',
 'tags': 'Paramount Pictures|Marvel Studios|Action|Adventure|Comicbook|Samuel L. Jackson|Robert Downey, Jr.|Chris Evans|Chris Hemsworth|Jeremy Renner|Scarlett Johansson|Tom Hiddleston|Mark Ruffalo|Clark Gregg',
 'filename': 'the-avengers_feature-trailer'}

In [37]:
import urllib.request
baseurl = r"https://www.traileraddict.com"
trailist = []

for i, h in enumerate(tdicts):
#     h = tdicts[0]
#     i = 0
    u = h['link']

    driver.get(baseurl + u)
    time.sleep(1)

    page = driver.page_source   
    #page = requests.get(baseurl + u).text
    soup = BeautifulSoup(page, "html.parser")

    x = soup.find('div', id="trailer_addict_video_frame")
    x = soup.find("video", id="trailerplayer_html5")
    h['mp4'] = x['src']
    h['i'] = i
    trailist.append(h)
    urllib.request.urlretrieve(x['src'], h['filename'] + '.mp4') 
    print(i,h)

0 {'imdb_id': 'tt0848228', 'film_title': 'The Avengers', 'year': 2012, 'tradd_url': '/the-avengers', 'link': '/the-avengers/feature-trailer', 'tags': 'Paramount Pictures|Marvel Studios|Action|Adventure|Comicbook|Samuel L. Jackson|Robert Downey, Jr.|Chris Evans|Chris Hemsworth|Jeremy Renner|Scarlett Johansson|Tom Hiddleston|Mark Ruffalo|Clark Gregg', 'filename': 'the-avengers_feature-trailer', 'mp4': 'https://vod-progressive.akamaized.net/exp=1614567052~acl=%2A%2F672628421.mp4%2A~hmac=67225923774e951b304981850cf239c0bc9193be72459ab42e5d0cafea9df44e/vimeo-prod-skyfire-std-us/01/4885/7/199427458/672628421.mp4?filename=The+Avengers+-+Feature+Trailer+%2853612%29.mp4', 'i': 0}
1 {'imdb_id': 'tt0472033', 'film_title': '9', 'year': 2009, 'tradd_url': '/9-feature', 'link': '/9-feature/trailer', 'tags': 'Focus Features|Action|CG Animation|Teaser|Remake', 'filename': '9-feature_trailer', 'mp4': 'https://vod-progressive.akamaized.net/exp=1614567460~acl=%2A%2F668367041.mp4%2A~hmac=229d266a30dc6e440

ContentTooShortError: <urlopen error retrieval incomplete: got only 5242880 out of 24369432 bytes>

In [38]:
df_trailist = pd.DataFrame(trailist)
df_trailist.to_csv("traileraddict_songlinks_downloaded.csv")

<div class="video-js2 vjs-fluid vjs-default-skin" id="trailerplayer">
</div>
